# Задание 1

В этом задании вам предстоит сделать так, чтобы TTS начал говорить другим голосом так, чтобы можно было контролировать интонации путем присвоения токена каждому слову.

**Важно!** Добавьте ссылку на свой форк с гитхаба в команду git clone

In [ ]:
!git clone https://github.com/Tikhon239/TTS_HW.git
!cd TTS_HW && curl gdrive.sh | bash -s https://drive.google.com/file/d/1ubAsUdWqHFY3MZJZVLqzxjH6lR_hKRyz/view?usp=sharing 
!cd TTS_HW && tar -xf libri_training_data.tar.gz

## Важно!

Следующая ячейка будет устанавливать библиотеку Apex, на колабе это может занять **12-20** минут! К сожалению, без этой библиотеки обучение данной реализации сети будет длиться слишком медленно, также из этой библиотеки используется оптимайзер FusedLamb, если же использовать стандартный Adam, обучение может стать нестабильным.

Если у вас есть своя локальная GPU, рекомендую использовать docker контейнер с Nvidia GPU Cloud - `nvcr.io/nvidia/pytorch:21.06-py3`, там эта библиотека уже предустановлена.

In [ ]:
try:
  import apex
except Exception:
  ! git clone https://github.com/NVIDIA/apex.git
  % cd apex
  !pip install --target=$nb_path --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" .
  %cd ..

In [ ]:
!pip install -r TTS_HW/requirements.txt
!pip install transformers
!pip install torchmetrics

In [ ]:
!cd TTS_HW && bash scripts/download_cmudict.sh
!cd TTS_HW && bash scripts/download_fastpitch.sh
!cd TTS_HW && bash scripts/download_waveglow.sh

### Описание задачи
Выше была склонирована оригинальная имплементация FastPitch [отсюда](https://github.com/NVIDIA/DeepLearningExamples/tree/master/PyTorch/SpeechSynthesis/FastPitch). В ней изменен лишь даталоадер - в батче помимо текста и целевой мелспектрограммы, также возвращаются токены `Prominencе`. Необходимо понять, как можно использовать эти токены и в какое место сети необходимо их добавить, чтобы затем, на инференсе, можно было контролировать интонации.

Так как мы сильно ограничены в ресурсах - обучать сеть с нуля не получится, необходимо лишь провести finetuning с оригинального чекпоинта от Nvidia, обученного на датасете LJSpeech. От вас требуется дообучить сеть на одном из голосов из датасета LibriTTS.

Все данные уже предобработаны и загружены, токены `Prominence` были извлечены с помощью [Wavelet Prosody Toolkit](https://github.com/asuni/wavelet_prosody_toolkit). Эти значения были квантизированы, в данных могут встречаться лишь значения 1, 2, 3. Эти значения извлечены для каждого слова и продублированы для каждой фонемы.

Из-за особенностей имплементации засовывать обучение в ноутбук крайне неудобно, поэтому предлагается изменить файлы исходного кода прямо открывая их в колабе (они видны в файл менеджере слева), а затем запустить обучение, выполнив ячейку ниже. Чтобы не потерять свои изменения, рекомендую, изменив файлы, просто коммитить их к себе в репозиторий прямо из колаба. Не забывайте это делать часто, так как колаб любит уничтожать вашу среду, даже если вы просто отошли попить кофе.

Уже 10 эпох обучения достаточно (около 15 минут обучения), чтобы была слышна разница. Mel loss должен опуститься где-то до значений 0.2-0.3, чтобы TTS звучал нормально. Так как данных очень мало, то если вы оставите сеть учиться надолго, скорее всего она переобучится, поэтому учить сильно дольше 20 эпох нет смысла.

In [ ]:
!cd TTS_HW && bash scripts/train.sh

После того как обучили сеть, прогоните ее на одной заданной фразе с разными значениями `Prominence`. Для этого в файле `inference.py` измените 384 строчку и запустите ячейку ниже. С разными значениями prominence фраза "What a day" должна произноситься с разной интонацией. Если вы этого добились, значит задание выполнено.

In [ ]:
!cd TTS_HW && bash scripts/inference_example.sh

In [ ]:
import IPython
#gen_kw = {'proms': torch.IntTensor([1,1,1,1,1,1,3,3,3,3]).to(b['text'].device)}
IPython.display.Audio('/content/TTS_HW/output/audio_phrase_1_64.txt/audio_0.wav')

In [ ]:
import IPython
#gen_kw = {'proms': torch.IntTensor([3,3,3,3,3,3,1,1,1,1]).to(b['text'].device)}
IPython.display.Audio('/content/TTS_HW/output/audio_phrase_1_64.txt/audio_0.wav')

In [ ]:
import IPython
#gen_kw = {'proms': torch.IntTensor([3,3,3,1,1,1,1,3,3,3]).to(b['text'].device)}
IPython.display.Audio('/content/TTS_HW/output/audio_phrase_1_64.txt/audio_0.wav')

# Задание 2 (5 баллов)

Это хорошо, что мы научились контролировать интонацию в произносимых фразах, но что делать на инференсе, ведь у нас нет этих значений `Prominence`? Можно попробовать научиться предсказывать их из текста. Для этого можно обучить какую-нибудь LM, при этом учить с нуля также не имеет смысла.

В задании предлагается заполнить отсутствующие ячейки ниже. Необходимый для зачета F1 score на тестовой выборке должен быть около `0.75`. Не забудьте про маски!

In [21]:
import torch
import transformers
import os
from pathlib import Path
import numpy as np
from torch.utils.data import Dataset, DataLoader
from transformers import BertForTokenClassification, BertTokenizer, BertConfig, BertModel
from transformers import AutoModelForTokenClassification, AutoTokenizer
from tqdm import tqdm
import torchmetrics

device='cuda:0'

In [22]:
class PromDataset(Dataset):
    def __init__(self, tokenizer, sentences, labels, max_len):
        self.len = len(sentences)
        self.sentences = sentences
        self.labels = labels
        self.max_len = max_len

        self.tokenizer = tokenizer(sentences,
                                   padding=True,
                                   truncation=True,
                                   max_length=max_len,
                                   return_tensors="pt")
        
    def __getitem__(self, index):
        ids = self.tokenizer.input_ids[index]
        mask = self.tokenizer.attention_mask[index]
        # [0] - cls token
        label = [0] + self.labels[index]
        label += [0] * (len(ids) - len(label))
        return {
            'input_ids': torch.tensor(ids, dtype=torch.long),
            'attention_mask': torch.tensor(mask, dtype=torch.long),
            'labels': torch.tensor(label, dtype=torch.long)
        } 
    
    def __len__(self):
        return self.len

In [26]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

sentences = []
label_proms = []
label_boundaries = []
for filename in Path('TTS_HW/libri_training_data').glob('*.prom'):
    with open(filename, "r") as f:
        words = []
        proms = []
        for line in f:
            word, prom = line.split("\t")[3:5]
            prom = int(np.clip(float(prom), 0, 2))

            num_repeat = len(tokenizer.encode(word, add_special_tokens=False))
            words.append(word)
            proms.extend([prom]*num_repeat)

        sentences.append(" ".join(words))
        label_proms.append(proms)

In [27]:
train_dataset = PromDataset(tokenizer, sentences[:-10], label_proms[:-10], 200)
test_dataset = PromDataset(tokenizer, sentences[-10:], label_proms[-10:], 200)

training_loader = DataLoader(train_dataset, batch_size=24, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=10, shuffle=False)

In [28]:
f1_criterion = torchmetrics.F1(num_classes=3).to(device)
acc_criterion = torchmetrics.Accuracy(num_classes=3).to(device)

model = BertForTokenClassification.from_pretrained('bert-base-cased', num_labels=3)
model.to(device)

optimizer = torch.optim.Adam(params=model.parameters(), lr=2e-05)
for epoch in range(10):
    print(f"EPOCH:{epoch}")
    t_bar = tqdm(training_loader)
    model.train()
    for batch in t_bar:
        batch = {k: v.to(device) for k, v in batch.items()}

        loss = model(**batch).loss

        optimizer.zero_grad()
        loss.backward()
        t_bar.set_description(desc=f'loss; {loss.item()}')
        optimizer.step()

    model.eval()
    with torch.inference_mode():
        accs = []
        f1_scores = []

        for batch in test_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            labels = batch["labels"]
            mask = batch["attention_mask"]
            masked_labels = labels.masked_select(mask == 1)

            out = model(**batch).logits.argmax(-1)
            masked_out = out.masked_select(mask == 1)
            
            accs.append(acc_criterion(masked_out, masked_labels))
            f1_scores.append(f1_criterion(masked_out, masked_labels))

    print(f"Accuracy: {torch.mean(torch.stack(accs)).item()}")
    print(f"F1 score: {torch.mean(torch.stack(f1_scores)).item()}")

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

EPOCH:0


  0%|          | 0/14 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
loss; 0.648619532585144: 100%|██████████| 14/14 [00:08<00:00,  1.59it/s]


Accuracy: 0.6885964870452881
F1 score: 0.6885964870452881
EPOCH:1


loss; 0.5320296287536621: 100%|██████████| 14/14 [00:08<00:00,  1.62it/s]


...

Accuracy: 0.7412280440330505
F1 score: 0.7412279844284058
EPOCH:8


loss; 0.18543954193592072: 100%|██████████| 14/14 [00:08<00:00,  1.61it/s]


Accuracy: 0.75
F1 score: 0.75
EPOCH:9


loss; 0.10350753366947174: 100%|██████████| 14/14 [00:08<00:00,  1.61it/s]

Accuracy: 0.7719298005104065
F1 score: 0.7719298005104065


Стоит отметить, что мы обучаем сеть на очень небольшом наборе данных - если же файнтюнить на полном ТТС датасете, то точность предсказаний меток может достичь 0.95, поэтому такую сеть можно использовать при реальном инференсе для более точного предсказания интонаций.

# Задание 3 (5 баллов) (опционально)
Если по какой-то причине что-то не получилось в предыдущих заданиях, то можно попробовать подготовить модель к инференсу. Задача - заэкспортировать код модели в ONNX, и проверить, что он работает.
Подсказка - вся модель скорее всего не заэкспортится сразу, стоит пробовать делать это по модулям, а если какая-то функция не поддерживается ONNX, то ее можно выделить отдельно и заэкспортировать, например, в TorchScript. На выходе ожидается, что есть набор onnx или torchscript модулей, выполнив последовательно которые можно получить тот же выход, что и при запуске модели на PyTorch.

Задание - по желанию, если не хочется делать что-то из того, что выше!